In [ ]:
import geopandas as gpd
import locale
import pandas as pd
import plotly.express as px

In [ ]:
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

In [ ]:
df = pd.read_csv('sources/results.csv')

In [ ]:
df['Mês'] = pd.to_datetime(df['month'], format = '%Y%m').dt.strftime('%b/%Y').str.capitalize()

In [ ]:
df = df.iloc[:, [0, 5, 1, 2, 3, 4]]

In [ ]:
df.columns = ['Mês (ordenação)', 'Mês', 'UF', 'Código', 'Assinaturas', 'Peso']

In [ ]:
gdf_simplified = gpd.read_file('sources/basemap-simplified.geojson')

In [ ]:
bounds = gdf_simplified.total_bounds

In [ ]:
fig_map = px.choropleth(
	df,
	geojson = gdf_simplified,
	locations = 'Código', # Coluna no DataFrame para cruzamento com o GeoJSON
    featureidkey = 'properties.code', # Chave no GeoJON para cruzmaneto com o DataFrame
    color = 'Peso',
	hover_name = 'UF',
	hover_data = {
		'Código': False,
		'Assinaturas': True,
		'Peso': ':.2f'
	},
	animation_frame = 'Mês',
	projection = 'equirectangular',
	color_continuous_scale = 'Reds',
	range_color=[df['Peso'].quantile(0.05), df['Peso'].quantile(0.95)]
)

fig_map.update_geos(
	fitbounds = 'locations',
	visible = False
)

fig_map.update_layout(
	coloraxis_colorbar = dict(
		title = 'Peso',
		tickformat = '.2f'
	)
)

fig_map.show()

In [ ]:
# fig.write_html('presentation.html', include_plotlyjs = 'cdn', full_html = True)

In [ ]:
fig_bar = px.bar(
	df,
	x = 'Assinaturas',
	y = 'UF',
	color = 'UF',
	animation_frame = 'Mês',
	orientation = 'h',
	opacity = .75,
	hover_data = {'Assinaturas': ':d'}
)

fig_bar.update_layout(
	yaxis = {'categoryorder': 'total ascending'},
	showlegend = False,
	bargap = 0.25
)

fig_bar.show()
